In [1]:
# Imports
from bertopic.representation import ZeroShotClassification
from bertopic import BERTopic
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

In [2]:
# Set up device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device set to {device}")
torch.set_default_device(device)

Device set to mps


In [5]:
import csv

# Loading Dataset
DATASET_PATH = "data/reddit_comments_2022_01_keywords_sample.csv"
with open(DATASET_PATH, 'r') as input_file:
    list_of_dict = list(csv.DictReader(input_file))
    reddit_df = pd.DataFrame.from_dict(list_of_dict)
    print(reddit_df.head(5)) # Display first 5 results
text = reddit_df["body"].values.tolist()

     link_id        subreddit ups downs  \
0  t3_rt375j            Games   0     0   
1  t3_rt3fyb            phish   0     0   
2  t3_rschai         Portland   0     0   
3  t3_rsj3pk       IceFishing   0     0   
4  t3_rssbks  disneylandparis   0     0   

                                                body  
0  There are two kinds of games in that 75-85 ran...  
1  We were supposed to be taking the train from t...  
2  &gt; that's a non-sequitur, maybe make it your...  
3  PM me if you'd like some recommendations. I'm ...  
4  I never got the Seven Dwarfs Mine train hype. ...  


In [41]:
# Calculate Embeddings of Reddit Text
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_model.to(device)
embeddings = embedding_model.encode(text,show_progress_bar=True)
embedding_path = "preprocessing/text_embeddings.pth" # Change here to move where output is
torch.save(embeddings, embedding_path)
print(f"Saved embeddings to {embedding_path}")

Batches:   0%|          | 0/2357 [00:00<?, ?it/s]

[[ 0.0553934   0.01637202 -0.0107393  ... -0.11356466 -0.07040752
  -0.00493962]
 [ 0.00164176 -0.02709313  0.04511569 ... -0.03688632 -0.06230261
  -0.02257638]
 [ 0.08276069  0.02041711  0.08701024 ...  0.05794099  0.02614011
   0.03917479]
 ...
 [-0.02895974 -0.09118299  0.07950093 ...  0.0930431  -0.08076359
   0.07564485]
 [-0.11080716  0.05790221  0.02361607 ... -0.09248571 -0.03076995
   0.00656432]
 [-0.042346   -0.06084095 -0.0260415  ... -0.08581486 -0.16089371
   0.02438405]]


In [6]:
topics = ["transit","not transit"]
model = ZeroShotClassification(topics, model="facebook/bart-large-mnli")
topic_model = BERTopic(representation_model=model)
embeddings = torch.load("preprocessing/text_embeddings.pth")

print(embeddings.shape)

topics, p = topic_model.fit_transform(documents=text[0:10], embeddings=embeddings[0:10])

df = pd.DataFrame(topics)
df["predictions"] = p

topic_out = "preprocessing/out/outputs.pth"
torch.save(topics,topic_out)

(75414, 384)


RuntimeError: Placeholder storage has not been allocated on MPS device!